In [5]:
%pip install beautifulsoup4
%pip install requests



In [6]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint

Creating the list we want into

In [7]:
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

Getting english titles from movies


In [8]:
headers = {'Accept-Language' : 'en-US, en;q=0.5'}

Setting the list of 250 moivies

In [11]:
pages = np.arange(1, 251, 50)
pages

array([  1,  51, 101, 151, 201])

 Storing each of the urls of 50 movies 

In [13]:
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

Creating Data Set

In [15]:
movies = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross})

movies.head(10)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,The Trial of the Chicago 7,(2020),129 min,7.9,76,"56,052",-
1,Tenet,(2020),150 min,7.8,69,"145,126",$53.80M
2,Spider-Man: Into the Spider-Verse,(2018),117 min,8.4,87,"351,964",$190.24M
3,The Gentlemen,(2019),113 min,7.8,51,"204,419",-
4,Harry Potter and the Sorcerer's Stone,(2001),152 min,7.6,64,"641,120",$317.58M
5,Knives Out,(2019),130 min,7.9,82,"414,864",$165.36M
6,Parasite,(2019),132 min,8.6,96,"506,882",$53.37M
7,Once Upon a Time... In Hollywood,(2019),161 min,7.6,83,"528,055",$142.50M
8,Soorarai Pottru,(2020),153 min,9.1,-,"29,663",-
9,1917,(2019),119 min,8.3,78,"389,434",$159.23M


In [16]:
movies.dtypes

movie             object
year              object
time_minute       object
imdb_rating      float64
metascore         object
vote              object
gross_earning     object
dtype: object

Cleaning the Data Set

In [17]:
# Cleaning 'year' column
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies.head(3)


,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,The Trial of the Chicago 7,2020,129 min,7.9,76,"56,052",-
1,Tenet,2020,150 min,7.8,69,"145,126",$53.80M
2,Spider-Man: Into the Spider-Verse,2018,117 min,8.4,87,"351,964",$190.24M


In [18]:
# Cleaning 'time_minute' column
movies['time_minute'] = movies['time_minute'].str.extract('(\d+)').astype(int)
movies.head(5)


,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,The Trial of the Chicago 7,2020,129,7.9,76,"56,052",-
1,Tenet,2020,150,7.8,69,"145,126",$53.80M
2,Spider-Man: Into the Spider-Verse,2018,117,8.4,87,"351,964",$190.24M
3,The Gentlemen,2019,113,7.8,51,"204,419",-
4,Harry Potter and the Sorcerer's Stone,2001,152,7.6,64,"641,120",$317.58M


In [19]:
# Cleaning 'metascore' column
movies['metascore'] = movies['metascore'].str.extract('(\d+)')
# convert it to float and if there are dashes turn it into NaN
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

In [20]:
# Cleaning 'vote' column
movies['vote'] = movies['vote'].str.replace(',', '').astype(int)
movies.head(3)


,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,The Trial of the Chicago 7,2020,129,7.9,76.0,56052,-
1,Tenet,2020,150,7.8,69.0,145126,$53.80M
2,Spider-Man: Into the Spider-Verse,2018,117,8.4,87.0,351964,$190.24M


In [ ]:
# Cleaning 'gross_earning' column
# left strip $ and right strip M 
movies['gross_earning'] = movies['gross_earning'].map(lambda x: x.lstrip('$').rstrip('M'))


In [26]:
# convert it to float and if there are dashes turn it into NaN
movies['gross_earning'] = pd.to_numeric(movies['gross_earning'], errors='coerce')
movies.head(5)


,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,The Trial of the Chicago 7,2020,129,7.9,76.0,56052,NaN
1,Tenet,2020,150,7.8,69.0,145126,53.80
2,Spider-Man: Into the Spider-Verse,2018,117,8.4,87.0,351964,190.24
3,The Gentlemen,2019,113,7.8,51.0,204419,NaN
4,Harry Potter and the Sorcerer's Stone,2001,152,7.6,64.0,641120,317.58


In [27]:
movies.dtypes

movie             object
year               int64
time_minute        int64
imdb_rating      float64
metascore        float64
vote               int64
gross_earning    float64
dtype: object

Final Data Set

In [28]:
movies


,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,The Trial of the Chicago 7,2020,129,7.9,76.0,56052,NaN
1,Tenet,2020,150,7.8,69.0,145126,53.80
2,Spider-Man: Into the Spider-Verse,2018,117,8.4,87.0,351964,190.24
3,The Gentlemen,2019,113,7.8,51.0,204419,NaN
4,Harry Potter and the Sorcerer's Stone,2001,152,7.6,64.0,641120,317.58
...,...,...,...,...,...,...,...
245,Into the Wild,2007,148,8.1,73.0,565568,18.35
246,Sin City,2005,124,8.0,74.0,735590,74.10
247,The Bourne Supremacy,2004,108,7.7,73.0,430415,176.24
248,Scent of a Woman,1992,156,8.0,59.0,258180,63.90


Saving the data to a csv file

In [32]:
movies.to_csv('movies.csv')